In [2]:
import glob
import logging
import os
import re
import subprocess
import sys
import random
from datetime import datetime

import numpy as np



from open_clip import create_model_and_transforms, trace_model, get_tokenizer
from training.file_utils import pt_load, check_exists, start_sync_process, remote_sync
from open_clip import create_model_and_transforms, trace_model, get_tokenizer

from tqdm import tqdm
import yaml
import os 
import random
import time
import json
LATEST_CHECKPOINT_NAME = "epoch_latest.pt"

In [3]:
model, preprocess_train, preprocess_val = create_model_and_transforms(
        "ViT-B-32",
        "openai",)
# checkpoint = pt_load('/home/mila/l/le.zhang/scratch/open_clip/src/Outputs/rank-discriminative-5e-06-weight0.2/checkpoints/epoch_5.pt', map_location='cpu')
# if 'epoch' in checkpoint:
#     # resuming a train checkpoint w/ epoch and optimizer state
#     start_epoch = checkpoint["epoch"]
#     sd = checkpoint["state_dict"]
#     if next(iter(sd.items()))[0].startswith('module'):
#         sd = {k[len('module.'):]: v for k, v in sd.items()}
#     model.load_state_dict(sd)
tokenizer=get_tokenizer("ViT-B-32")

In [6]:
tokenizer('###')

tensor([[49406,     2, 15483, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]])

In [ ]:
from open_clip.tokenizer import SimpleTokenizer
import torch
_tokenizer = SimpleTokenizer()
# def decode(output_ids: torch.Tensor):
#     output_ids = output_ids.cpu().numpy()
#     return _tokenizer.decode(output_ids)
_tokenizer.decode(tokenizer('###'))

In [35]:
_tokenizer.decoder[49407]

'<end_of_text>'

In [105]:
from vl_checklist.data_loader import DataLoader
config_file="/home/mila/l/le.zhang/scratch/open_clip/src/VL-CheckList/configs/sample.yaml"
m = yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader)
batch_size = m["BATCH_SIZE"]
model = model                   
max_num = m["MAX_NUM"]
data_names = m["DATA"]["TEST_DATA"]
task = m["TASK"]
types = m["DATA"]["TYPES"]
dir = m["OUTPUT"]["DIR"]
sample_num = m["OUTPUT"]["NUM"]
data_type=types[0]


data/Attribute/vg/color.json


In [116]:
import torch
from PIL import Image
results = {}
index = 0
batch_size=8
cur_dir="./"
device='cuda'
model.to(device)
results={}
max_number=None
for 
d= DataLoader(data_names, data_type, task)
if task == 'itm':
    for name in d.data:
        results[name]=[]
        sample_true = []
        sample_false = []
        # true false positive negative
        tp, tn, fp, fn = 0, 0, 0, 0
        if max_number:
            d.data[name] = d.data[name][:max_number]
        len_data=len(d.data[name])//2
        starttime = time.time()
        for batch in tqdm(chunks(d.data[name], batch_size), desc="Progress", ncols=100,
                        total=int(len(d.data[name]) / batch_size)):
            images = [z["path"] for z in batch]
            images=images[::2]
            texts = [z["texts"][0] for z in batch]
            b=2
            t=[]
            for i in range(0,len(texts),2):
               t.append(texts[i:i+b])
            # texts = [random.choice(z["texts"]) for z in batch]
            labels = [z["label"] for z in batch]
            for j in range(len(images)):
                image = preprocess_train(Image.open(images[j])).unsqueeze(0).to(device)
                # text format is [["there is a cat","there is a dog"],[...,...]...]
                text = tokenizer(t[j]).to(device)

                with torch.no_grad():
                
                    image_features = model.encode_image(image)
                    text_features = model.encode_text(text)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    text_features /= text_features.norm(dim=-1, keepdim=True)

                    logits_per_image = (100.0 * image_features @ text_features.T).softmax(dim=-1)
                    prob=logits_per_image.softmax(dim=-1).cpu().numpy()[0]
                    results[name].append(1 if prob[0]>prob[1] else 0)
        endtime = time.time()
   
       
      

Progress: 2893it [03:49, 12.63it/s]                                                                 


In [114]:
sum(results['vg'])/len_data

0.6254646036822543

In [117]:
sum(results['vg'])/len_data

0.6681649235024635

In [5]:
A=None


In [1]:
import torch

In [6]:
a=torch.tensor([3,100,2,1])

In [7]:
torch.clamp(a,0,5)

tensor([3, 5, 2, 1])